In [16]:
import pandas as pd
df = pd.read_csv('encoded.csv')
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error as mape


# Assuming X contains your feature data with 97 columns and y contains the target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(97,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model to the training data
model.fit(X_train_scaled, y_train, epochs=350, batch_size=64)

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)

Epoch 1/350
113/113 [==============================] - 0s 747us/step - loss: 4174.5591
Epoch 2/350
113/113 [==============================] - 0s 784us/step - loss: 436.0879
Epoch 3/350
113/113 [==============================] - 0s 793us/step - loss: 394.2020
Epoch 4/350
113/113 [==============================] - 0s 828us/step - loss: 370.1440
Epoch 5/350
113/113 [==============================] - 0s 813us/step - loss: 349.6381
Epoch 6/350
113/113 [==============================] - 0s 821us/step - loss: 336.0693
Epoch 7/350
113/113 [==============================] - 0s 837us/step - loss: 322.2716
Epoch 8/350
113/113 [==============================] - 0s 825us/step - loss: 313.1203
Epoch 9/350
113/113 [==============================] - 0s 804us/step - loss: 298.4767
Epoch 10/350
113/113 [==============================] - 0s 818us/step - loss: 292.8786
Epoch 11/350
113/113 [==============================] - 0s 813us/step - loss: 279.3585
Epoch 12/350
113/113 [=============================

In [23]:
# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared (R2) Score
r2 = r2_score(y_test, y_pred)

accuracy = (1-mape(y_test, y_pred))*100

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)
print("Accuracy :", accuracy)

Mean Squared Error (MSE): 471.97675452758193
Root Mean Squared Error (RMSE): 21.725025996016253
Mean Absolute Error (MAE): 15.925787395320425
R-squared (R2) Score: 0.5503141941574949
Accuracy : 87.04025687786817


In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers.legacy import Adam
from keras import regularizers

# Assuming you have your data stored in 'X' (input features) and 'y' (target variable)
# X.shape should be (number of samples, 97)
# y.shape should be (number of samples,)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the function to create the CNN model
def create_model(filters, kernel_size, pool_size, hidden_nodes, learning_rate, dropout_rate, regularization):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(97, 1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(hidden_nodes, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    # Define the optimizer with the specified learning rate
    optimizer = Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    return model

# Define the hyperparameters to try
filters_list = [32, 64]  # Number of filters in the convolutional layer
kernel_size_list = [3, 5]  # Size of the convolutional kernel
pool_size_list = [2, 4]  # Size of the max pooling window
hidden_nodes_list = [64, 128]  # Number of nodes in the dense layer
learning_rate_list = [0.001, 0.01]  # Learning rates
dropout_rate_list = [0.2, 0.5]  # Dropout rates
regularization_list = [0.001, 0.01]  # Regularization strengths

results = []

# Loop over all combinations of hyperparameters
for filters in filters_list:
    for kernel_size in kernel_size_list:
        for pool_size in pool_size_list:
            for hidden_nodes in hidden_nodes_list:
                for learning_rate in learning_rate_list:
                    for dropout_rate in dropout_rate_list:
                        for regularization in regularization_list:
                            # Create the model
                            model = create_model(filters, kernel_size, pool_size, hidden_nodes,
                                                 learning_rate, dropout_rate, regularization)

                            # Train the model
                            model.fit(np.expand_dims(X_train, axis=-1), y_train, epochs=50, batch_size=32, verbose=0)

                            # Evaluate the model on the test set
                            y_pred = model.predict(np.expand_dims(X_test, axis=-1))

                            # Calculate metrics
                            mae = mean_absolute_error(y_test, y_pred)
                            mse = mean_squared_error(y_test, y_pred)
                            rmse = np.sqrt(mse)
                            r2 = r2_score(y_test, y_pred)
                            accuracy = (1 - (mae / np.mean(y_test))) * 100

                            # Store the results in a dictionary
                            result = {
                                'Filters': filters,
                                'Kernel Size': kernel_size,
                                'Pool Size': pool_size,
                                'Hidden Nodes': hidden_nodes,
                                'Learning Rate': learning_rate,
                                'Dropout Rate': dropout_rate,
                                'Regularization': regularization,
                                'MAE': mae,
                                'MSE': mse,
                                'RMSE': rmse,
                                'R2 Score': r2,
                                'Accuracy': accuracy
                            }

                            results.append(result)

# Create a DataFrame to tabulate the results
results_df = pd.DataFrame(results)


70/70 [==============================] - 0s 713us/step


In [35]:
results_df

,Filters,Kernel Size,Pool Size,Hidden Nodes,Learning Rate,Dropout Rate,Regularization,MAE,MSE,RMSE,R2 Score,Accuracy
0,32,3,2,64,0.001,0.2,0.001,16.916868,475.652976,21.809470,0.549857,87.097634
1,32,3,2,64,0.001,0.2,0.010,16.922850,477.625880,21.854654,0.547990,87.093072
2,32,3,2,64,0.001,0.5,0.001,17.253256,500.699820,22.376323,0.526153,86.841073
3,32,3,2,64,0.001,0.5,0.010,17.145302,492.925925,22.201935,0.533510,86.923409
4,32,3,2,64,0.010,0.2,0.001,17.521687,517.599449,22.750812,0.510160,86.636344
...,...,...,...,...,...,...,...,...,...,...,...,...
123,64,5,4,128,0.001,0.5,0.010,16.926272,484.099201,22.002254,0.541864,87.090462
124,64,5,4,128,0.010,0.2,0.001,16.727368,485.010644,22.022957,0.541001,87.242164
125,64,5,4,128,0.010,0.2,0.010,19.460959,625.681914,25.013635,0.407874,85.157275
126,64,5,4,128,0.010,0.5,0.001,17.416902,493.345142,22.211374,0.533113,86.716262
